In [5]:
from flask import Flask, request, render_template
import os
import cv2
import torch
from torchvision.ops.boxes import batched_nms
import datetime
import hashlib
import json

In [7]:
class Detect:
    def __init__(self, img, classPath=r'D:\Projects\Python\mushroom/classes.txt' ,modelPath=r'D:\Projects\Python\mushroom/weights/best.onnx'):
        self.classes = open(classPath).read().split("\n")  # 读取类别
        self.model = cv2.dnn.readNetFromONNX(modelPath)  # 读取模型
        self.img = cv2.imread(img)  # 读取图片
        self.originX = self.img.shape[1]  # 原始宽度
        self.originY = self.img.shape[0]  # 原始长度
        
    def preProcessing(self):
        self.img = cv2.resize(self.img, (640, 640))  # 裁剪大小
        blob = cv2.dnn.blobFromImage(self.img, 1/255.0, swapRB=True)  # 由图片加载数据，这里还可以进行缩放、归一化等预处理
        return blob

    def run(self, blob):
        self.model.setInput(blob)  # 输入图像
        outs = self.model.forward(self.model.getUnconnectedOutLayersNames())[-1][0]  # 前向推理，-1表示最后一层，取后[1,25200,10]
        return outs

    def afterProcessing(self, outs):
        boxes_list = []  # 监测框
        classes_list = []  # 类别列表
        score_list = []  # 打分
        scaleX = self.originX / 640
        scaleY = self.originY / 640
        for out in outs:
            if out[4] > 0.8:
                x1 = int((out[0] - 0.5 * out[2]) * scaleX)  # 左上
                y1 = int((out[1] - 0.5 * out[3]) * scaleY)  # 左上
                x2 = int((out[0] + 0.5 * out[2]) * scaleX)  # 右下
                y2 = int((out[1] + 0.5 * out[3]) * scaleY)  # 右下
                classes_scores = out[5:]  # 各种类别的分数
                _, _, _, max_idx = cv2.minMaxLoc(classes_scores)  # 最高分数的类别
                class_id = max_idx[1]  # 取类别id
                boxes_list.append([x1, y1, x2, y2])  # 加入矩形框列表
                classes_list.append(class_id)  # 加入类别列表
                score_list.append(classes_scores[class_id])  # 加入分数列表
        boxes_list_tensor = torch.Tensor(boxes_list)  # 将矩形框转为张量
        classes_list_tensor = torch.Tensor(classes_list)  # 将类别转为张量
        score_list_tensor = torch.Tensor(score_list)  # 将分数转为张量
        nms_list = batched_nms(boxes_list_tensor, score_list_tensor, classes_list_tensor, 0.001)  # nms
        final_box = []
        final_class = []
        final_score = []
        for index in nms_list:
            index = int(index)  # tensor -> int
            final_box.append(boxes_list[index])  # 获取矩形框坐标
            final_class.append(self.classes[classes_list[index]])  # 获取类别
            final_score.append(f'{score_list[index]:.2%}')  # 获取置信度
        return final_box, final_class, final_score

app = Flask(__name__)
# 处理文件上传请求
@app.route('/upload', methods=['POST'])
def upload_file():
    img_bytes = request.get_data()
    if img_bytes:
        md5 = hashlib.md5(img_bytes).hexdigest()
        # 写入文件
        date = str(datetime.datetime.now()).split(' ')
        newfilename = date[0] + '-' + date[1].replace(':', '.') + '-' + md5 + '.jpg'
        print(newfilename)
        with open(newfilename, 'wb') as f:
            f.write(img_bytes)
        detect = Detect(newfilename)
        blob = detect.preProcessing()  # 预处理图像
        outs = detect.run(blob)  # 前向推理
        final_box, final_class, final_score = detect.afterProcessing(outs)  # 后处理
        result = {"objects":[]}
        for i in range(len(final_class)):
            result["objects"].append({
                "class": final_class[i], 
                "confidence": final_score[i],
                "box": final_box[i]
            })
        jsonstr = json.dumps(result)
        print(jsonstr)
    return jsonstr

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.31.44:5000
Press CTRL+C to quit


2023-05-19-00.10.28.880699-43adf46e9248cf3f8732c5489c26d7de.jpg


192.168.31.174 - - [19/May/2023 00:10:29] "POST /upload HTTP/1.1" 200 -


{"objects": [{"class": "Amanita muscaria", "confidence": "99.96%", "box": [2239, 3887, 2600, 4094]}, {"class": "Amanita muscaria", "confidence": "99.88%", "box": [1842, 3900, 2102, 4099]}, {"class": "Amanita muscaria", "confidence": "99.80%", "box": [2752, 3888, 2952, 4099]}]}
2023-05-19-00.10.45.246022-0ca7d8c3cdbff3bec3f84612f3784e7a.jpg


...

2023-05-19-00.41.36.007542-7d9ac5a27969c40a102db3aef98a5697.jpg
{"objects": [{"class": "Aleuria aurantia", "confidence": "99.98%", "box": [433, 267, 639, 477]}, {"class": "Aleuria aurantia", "confidence": "99.98%", "box": [3, 16, 336, 333]}]}


192.168.31.174 - - [19/May/2023 00:41:36] "POST /upload HTTP/1.1" 200 -


2023-05-19-00.41.43.436764-2327430b8e7ae8e6a91caf70d3d14339.jpg


192.168.31.174 - - [19/May/2023 00:41:43] "POST /upload HTTP/1.1" 200 -


{"objects": []}
